In [1]:
import pandas as pd
import os
from langdetect import detect, LangDetectException
import re
import emoji

In [2]:
# Load the Raw CSV Files

input_folder = "../scraped_reviews"
all_data = []

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(input_folder, file))
        all_data.append(df)

raw_df = pd.concat(all_data, ignore_index=True)
print(f"Total records before cleaning: {len(raw_df)}")

Total records before cleaning: 11439


In [3]:
# Remove rows with missing reviews or ratings
clean_df = raw_df.dropna(subset=['review_text', 'rating'])

# Remove duplicates based on text and bank
clean_df = clean_df.drop_duplicates(subset=['review_text', 'bank_name'])

print(f"Total records after cleaning: {len(clean_df)}")

Total records after cleaning: 4945


In [4]:
def is_english(text):
    try:
        # Check if text is not empty and is English
        if not text or not isinstance(text, str) or text.isspace():
            return False
        return detect(text) == 'en'
    except LangDetectException:
        return False

def has_only_emojis_or_special_chars(text):
    if not text or not isinstance(text, str):
        return True
    
    # Remove all emojis
    text_without_emojis = emoji.replace_emoji(text, replace='')
    
    # Remove all special characters and whitespace
    text_clean = re.sub(r'[^a-zA-Z0-9\s]', '', text_without_emojis).strip()
    
    return len(text_clean) == 0

# First filter for English reviews
english_mask = clean_df['review_text'].apply(is_english)
df_english = clean_df[english_mask].copy()

# Then filter out emoji/special character only rows
valid_text_mask = ~df_english['review_text'].apply(has_only_emojis_or_special_chars)
df_clean = df_english[valid_text_mask].copy()

print(f"Original shape: {clean_df.shape}")
print(f"After English filter: {df_english.shape}")
print(f"Final clean shape: {df_clean.shape}")

Original shape: (4945, 5)
After English filter: (3402, 5)
Final clean shape: (3402, 5)


In [5]:
clean_df = df_clean.copy()

In [6]:
# Convert date to standard YYYY-MM-DD
clean_df['date'] = pd.to_datetime(clean_df['date'], errors='coerce')
clean_df = clean_df.dropna(subset=['date'])  # Drop rows with invalid dates
clean_df['date'] = clean_df['date'].dt.strftime('%Y-%m-%d')

In [7]:
clean_df = clean_df[['review_text', 'rating', 'date', 'bank_name', 'source']]

In [8]:
print(clean_df.sample(10))
print(clean_df.columns)
print(len(clean_df))

                                            review_text  rating        date  \
2527                                     It is best app       5  2025-02-13   
5756  Good actually , but it have to WORK when devel...       5  2025-04-21   
4146                         It is good and Okay thanks       5  2024-02-29   
245                                 The worest app ever       1  2024-10-03   
5921      Excellent UI/UX and beyond Banking services👍👍       5  2025-01-30   
480                 Worest app, it cannot be downloaded       2  2024-06-05   
5427  It's not working at abroad for me. It was good...       1  2023-03-14   
4517  The new version is useless it doesn't work at ...       1  2024-01-24   
3203                           Which is very important!       5  2024-10-10   
3271                                   Its eassy to use       5  2024-09-23   

                        bank_name       source  
2527  Commercial Bank of Ethiopia  Google Play  
5756                  Dashen Ban

In [9]:
# Save the Clean CSV
clean_df.to_csv("../scraped_reviews/cleaned_reviews.csv", index=False)
print("✅ Cleaned dataset saved as 'cleaned_reviews.csv'")

✅ Cleaned dataset saved as 'cleaned_reviews.csv'
